In [51]:
# Import Required Packages
import torch
import os
import json
import sys
import re
import random
import importlib.util
from typing import *
from tqdm import tqdm 
from typing import List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap


from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from datasets import Dataset


from JS_Architects import *

# Summary

We will use Architects model

1) Implement Custom pipeline, no prompting just prediction with shrekking of the embedding 

2) Automated Forsequence Classification
    1) No prompting
    2) Prompt: "Classify with labels encode in binary"
    3) Prompt: "Clasify with list label: ["depth",...]"
    4) Prompt "Classify with list label: ["depth",...]. Depth represents..., Containment represents...."

# Custom Pipeline 

## 1. Prepare Data

### Load 


In [52]:
with open("perceptions_training.json", "r") as f:
    dic_training = json.load(f)

with open("perceptions_testing.json", "r") as f:
    dic_testing = json.load(f)

label_list = [
    "containment",
    "depth",
    "symmetry",
    "categorical",
    "spatial-Orientation",
    "spatial-Ordinal",
    "similarity",
    "quantitative",
    "replication",
    "figure-Ground",
    "continuity",
    "size",
    "closure",
    "centroid",
    "topological",
    "motion",
]

### Format

In [53]:
from datasets import Dataset

def prepare_data_for_multilabel_classification(
    dic_training,
    instruction,
    label_list,
    inp_prefix="<I>",
    out_prefix="<O>",
    arr_sep="\n",
    exa_sep="\n---\n",
    eos_token="<EOS>"
):
    llama_data = []

    # Create a mapping from label to index
    label_to_index = {label: i for i, label in enumerate(label_list)}

    for entry_id, content in dic_training.items():
        # Extract perceptions (labels) and encode them as a binary vector
        perceptions = content.get("perceptions", [])
        label_vector = labels_to_binary(label_list, perceptions)

        # Combine train and test examples
        examples = content.get("example", {}).get("train", []) + content.get("example", {}).get("test", [])

        # Format examples into a single input string
        formatted_examples = []
        for example in examples:
            input_data = f"{inp_prefix}{format_array(example['input'], arr_sep)}"
            output_data = f"{out_prefix}{format_array(example['output'], arr_sep)}{eos_token}"
            formatted_examples.append(f"{input_data}{exa_sep}{output_data}")

        # Combine all examples into one input text
        combined_text = exa_sep.join(formatted_examples)

        # Add the structured data for fine-tuning
        llama_data.append({
            "instruction": f"{instruction}",
            #"instruction": f"{instruction}: {', '.join(label_list)}.",
            "input": combined_text,
            "output": label_vector,  # Multi-label as binary vector
        })

    return llama_data

def format_array(array, arr_sep="\n"):
    """
    Helper function to format a 2D array into a string with row-wise separation.
    """
    return arr_sep.join([" ".join(map(str, row)) for row in array])

def labels_to_binary(label_list, input_labels):
    """
    Convert perceptions into a binary vector based on the label list.
    Handles both single strings and lists of strings for input_labels.
    """
    # Ensure input_labels is treated as a list
    if isinstance(input_labels, str):
        input_labels = [input_labels]
    
    # Create a set of lowercase input labels
    input_set = set(label.lower() for label in input_labels)
    
    # Generate the binary vector
    return [1 if label.lower() in input_set else 0 for label in label_list]



In [54]:
instruction0 = ""
instruction1 = "Classify the relationship between the input and output sequences based on perceptions"

# List
llama_data = prepare_data_for_multilabel_classification(dic_training,instruction0,label_list)

# Dict
llama_data_dict = {
    "instruction": [item["instruction"] for item in llama_data],
    "input": [item["input"] for item in llama_data],
    "output": [item["output"] for item in llama_data],
}

# Dataset
llama_data_dataset = Dataset.from_dict(llama_data_dict)

llama_data_dataset[0]


{'instruction': '',
 'input': '<I>0 7 7\n7 7 7\n0 7 7\n---\n<O>0 0 0 0 7 7 0 7 7\n0 0 0 7 7 7 7 7 7\n0 0 0 0 7 7 0 7 7\n0 7 7 0 7 7 0 7 7\n7 7 7 7 7 7 7 7 7\n0 7 7 0 7 7 0 7 7\n0 0 0 0 7 7 0 7 7\n0 0 0 7 7 7 7 7 7\n0 0 0 0 7 7 0 7 7<EOS>\n---\n<I>4 0 4\n0 0 0\n0 4 0\n---\n<O>4 0 4 0 0 0 4 0 4\n0 0 0 0 0 0 0 0 0\n0 4 0 0 0 0 0 4 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 4 0 4 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 4 0 0 0 0<EOS>\n---\n<I>0 0 0\n0 0 2\n2 0 2\n---\n<O>0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 2\n0 0 0 0 0 0 2 0 2\n0 0 0 0 0 0 0 0 0\n0 0 2 0 0 0 0 0 2\n2 0 2 0 0 0 2 0 2<EOS>\n---\n<I>6 6 0\n6 0 0\n0 6 6\n---\n<O>6 6 0 6 6 0 0 0 0\n6 0 0 6 0 0 0 0 0\n0 6 6 0 6 6 0 0 0\n6 6 0 0 0 0 0 0 0\n6 0 0 0 0 0 0 0 0\n0 6 6 0 0 0 0 0 0\n0 0 0 6 6 0 6 6 0\n0 0 0 6 0 0 6 0 0\n0 0 0 0 6 6 0 6 6<EOS>\n---\n<I>2 2 2\n0 0 0\n0 2 2\n---\n<O>2 2 2 2 2 2 2 2 2\n0 0 0 0 0 0 0 0 0\n0 2 2 0 2 2 0 2 2\n0 0 0 0 0 0 0 0 0\n0 0 0 0 0

In [55]:
llama_data_list_testing = prepare_data_for_multilabel_classification(dic_testing,instruction0, label_list)

# Restructure llama_data
llama_data_dict_testing = {
    "instruction": [item["instruction"] for item in llama_data_list_testing],
    "input": [item["input"] for item in llama_data_list_testing],
    "output": [item["output"] for item in llama_data_list_testing],
}

llama_data_dataset_testing = Dataset.from_dict(llama_data_dict_testing)
llama_data_dataset_testing[10]

{'instruction': '',
 'input': '<I>0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0\n0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0\n0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0\n0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0\n0 0 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 0 0 0\n0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1\n0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1\n0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1\n0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0\n0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0

## 2. Load Model and Tokenizer

In [56]:
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig
from datasets import Dataset


# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("nvidia/Mistral-NeMo-Minitron-8B-Base")
model = AutoModelForCausalLM.from_pretrained("nvidia/Mistral-NeMo-Minitron-8B-Base")

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.01it/s]


In [57]:
print(f"The initial length vocabulary: {len(tokenizer.vocab)}")

The initial length vocabulary: 131072


## 3. Shrink and FT the tokenizer

In [58]:
def build_corpus_for_shrinking(hf_dataset):
    """
    Concatenate the 'input' + 'instruction' from the dataset
    to ensure all relevant tokens appear.
    """
    corpus_list = []
    for sample in hf_dataset:
        text = (sample["instruction"] or "") + " " + (sample["input"] or "")
        corpus_list.append(text)
    # Combine into one big string
    corpus = "\n".join(corpus_list)
    return corpus

corpus = build_corpus_for_shrinking(llama_data_dataset)

len(corpus)

376186

In [59]:
model, tokenizer, _ = prepare_model(
        model=model,        # Path or HF ID, e.g. "llama-7b-hf"
        mode="transformers",          # or another supported mode
        tokenizer=tokenizer,               # let prepare_model create it or pass your own
        shrink_embedding=False,       # We'll handle shrinking below
        #device_map=device_map,
        # Additional kwargs if needed
        #**tokenizer_kwargs
    )

shrink_embeddings(
            model=model,
            tokenizer=tokenizer,
            corpus=corpus,                  # ensures relevant tokens are kept
            keep_special_tokens=True,
            keep_normalizer=False,
            keep_token_order=True
        )

print("Tokenizer size after shrinking:", len(tokenizer.vocab))


Tokenizer size after shrinking: 32


As we shrinked the embedding we will fastly FT the model

In [60]:
from peft import get_peft_model, LoraConfig

peft_configs = [
    {
        "r": 32,
        "lora_alpha": 64,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.1,
        "bias": "none",
        "task_type": "CAUSAL_LM"
    }
]

model_shrinked = get_peft_model(model, LoraConfig(**peft_configs[0]))

model_shrinked

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32, 4096)
        (layers): ModuleList(
          (0-39): 40 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Line

In [61]:
peft_configs = [
    {
        "r": 32,
        "lora_alpha": 64,
        "target_modules": ["q_proj", "v_proj"],
        "lora_dropout": 0.1,
        "bias": "none",
        "task_type": "CAUSAL_LM"
    }
]

model, tokenizer, _ = prepare_model(
        model=model,        # Path or HF ID, e.g. "llama-7b-hf"
        mode="transformers",          # or another supported mode
        formatter= None,
        tokenizer=tokenizer,               # let prepare_model create it or pass your own
        shrink_embedding=True,       # We'll handle shrinking below
        dequantize= False,
        peft= peft_configs,
        #device_map=device_map,
        # Additional kwargs if needed
        #**tokenizer_kwargs
    )

*** Shrink embedding...


AttributeError: 'NoneType' object has no attribute 'get_corpus'

In [65]:
# Step 2: Define the model with a classification head
class LLMWithClassificationHead(nn.Module):
    def __init__(self, base_model, num_labels):
        super().__init__()
        self.base_model = base_model
        self.num_labels = num_labels
        hidden_size = base_model.config.hidden_size

        # A linear head for classification with num_labels outputs
        self.classifier = nn.Linear(hidden_size, num_labels)
    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        input_ids: (batch_size, seq_len)
        labels:    (batch_size, num_labels), 0/1 multi-label
        """
        # 1) Forward pass through LLaMA in "causal LM" mode
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )
        # 2) Extract the last hidden state: (batch_size, seq_len, hidden_size)
        last_hidden = outputs.hidden_states[-1]

        # 3) We'll use the hidden state at the last token as the "pooled" representation
        pooled = last_hidden[:, -1, :]  # (batch_size, hidden_size)

        # 4) Classification head to produce (batch_size, num_labels)
        logits = self.classifier(pooled)

        # 5) If we have labels, compute BCE with logits (multi-label classification)
        loss = None
        if labels is not None:
            # shape of labels should be (batch_size, num_labels)
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())

        return {
            'logits': logits,
            'loss': loss,
            'hidden_states': outputs.hidden_states,
            'attentions': outputs.attentions if hasattr(outputs, 'attentions') else None
        }

In [66]:
# Instantiate your custom model
num_labels = 16
model_Classification = LLMWithClassificationHead(
    base_model=model,
    num_labels=num_labels
)

model_Classification

LLMWithClassificationHead(
  (base_model): MistralForCausalLM(
    (model): MistralModel(
      (embed_tokens): Embedding(32, 4096)
      (layers): ModuleList(
        (0-39): 40 x MistralDecoderLayer(
          (self_attn): MistralSdpaAttention(
            (q_proj): lora.Linear(
              (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=32, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=32, out_features=4096, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_magnitude_vector): ModuleDict()
            )
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
         

In [71]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels from inputs
        labels = inputs.pop("labels")

        # Get logits from the model
        logits = model(**inputs)

        # Define Binary Cross-Entropy Loss with logits
        loss_fn = nn.BCEWithLogitsLoss()

        # Compute loss
        loss = loss_fn(logits, labels)
        return (loss, logits) if return_outputs else loss

def tokenize_function(example):
    tokenized = tokenizer(
        example["input"],
        #padding="max_length",
        truncation=True,
        max_length=8192,
        return_tensors="pt"
    )
    return {
        "input_ids": tokenized["input_ids"].squeeze().tolist(),
        "attention_mask": tokenized["attention_mask"].squeeze().tolist(),
        "labels": torch.tensor(example["output"], dtype=torch.float32).tolist()  # Ensure labels are floats
    }

class DataCollatorWithLabels:
    def __call__(self, features):
        input_ids = torch.stack([torch.tensor(f["input_ids"]) for f in features])
        attention_mask = torch.stack([torch.tensor(f["attention_mask"]) for f in features])
        labels = torch.stack([torch.tensor(f["labels"], dtype=torch.float32) for f in features])  # Stack labels
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}



In [72]:
# Instantiate your custom model
model_Classification = LLMWithClassificationHead(base_model=model, num_labels=16)

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    evaluation_strategy="steps",  # Evaluate periodically
    eval_steps=500,              # Evaluate every 500 steps
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,  # Reduced
    num_train_epochs=3,
    save_strategy="epoch",
    save_total_limit=2,
    seed=42,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    logging_dir="./logs",
    logging_steps=50,
    report_to="tensorboard",
    fp16=True,                    # Mixed-precision
    push_to_hub=False,
    ddp_find_unused_parameters=False,
)

# Apply tokenization
tokenized_dataset = llama_data_dataset.map(tokenize_function, batched=True)
tokenized_dataset_testing = llama_data_dataset_testing.map(tokenize_function, batched=True)

data_collator = DataCollatorWithLabels()

/home/jdelinea/anaconda3/envs/BarcHandbook/lib/python3.10/site-packages/transformers/training_args.py:1539: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/150 [00:00<?, ? examples/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [70]:
# Define trainer
trainer = CustomTrainer(
    model=model_Classification,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_testing,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

NameError: name 'tokenized_dataset' is not defined

In [ ]:
# Step 3: Training loop
model_name = "<path_to_base_model>"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LLMWithClassificationHead(model_name, num_labels=16).to("cuda")

train_data = [{"input": "...", "label": [0, 1, 0, 0]}, {"input": "...", "label": [1, 0, 0, 0]}]
train_dataset = ClassificationDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# Training
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")
        labels = batch["labels"].to("cuda")

        outputs = model(input_ids, attention_mask, labels=labels)
        loss = outputs["loss"]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
